In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pypfopt
from data import *

Excel file format cannot be determined, you must specify an engine manually.
login success!


Exception: 抱歉，您每天最多访问该接口50次，权限的具体详情访问：https://tushare.pro/document/1?doc_id=108。

In [5]:
from pypfopt import black_litterman, risk_models
from pypfopt import BlackLittermanModel, plotting

In [2]:
prices

NameError: name 'prices' is not defined

In [7]:
S = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
#delta = black_litterman.market_implied_risk_aversion(market_prices)
#delta

In [11]:
np.series(market_prices)

AttributeError: module 'numpy' has no attribute 'series'

In [16]:
delta = black_litterman.market_implied_risk_aversion(market_prices)

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [1]:
from RiskParity import *

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

def risk_parity(covariance_matrix, asset_names):
    """
    Computes the weights of a risk parity portfolio.

    Parameters:
    - covariance_matrix (np.ndarray or pd.DataFrame): A square matrix of asset returns covariances.
    - asset_names (list): A list of asset names, corresponding to the rows and columns of the covariance matrix.

    Returns:
    - weights (dict): A dictionary of asset weights, with asset names as keys and weights as values.
    """
    # Convert the covariance matrix to a numpy array if it's a DataFrame
    if isinstance(covariance_matrix, pd.DataFrame):
        covariance_matrix = covariance_matrix.values

    # Define the objective function for risk parity optimization
    def objective_function(weights, covariance_matrix):
        # Calculate the portfolio risk
        portfolio_risk = np.sqrt(weights @ covariance_matrix @ weights.T)

        # Calculate the contribution of each asset to the portfolio risk
        MRC = covariance_matrix @ weights.T
        # Risk Contribution
        RC = np.multiply(MRC,weights.T) / portfolio_risk

        # Calculate the difference between the target and actual risk contributions
        risk_contribution_diff = RC - 1 / covariance_matrix.shape[0]

        # Calculate the sum of squared differences
        sum_of_squares = np.sum(np.square(risk_contribution_diff))

        return sum_of_squares

    
    def total_weight_constraint(x):
        return np.sum(x)-1.0

    def long_only_constraint(x):
        return x

    cons = ({'type': 'eq', 'fun': total_weight_constraint},
    {'type': 'ineq', 'fun': long_only_constraint})
    
    # Define the initial guess for the weights (equal weighting)
    n_assets = covariance_matrix.shape[0]
    initial_weights = np.array([1 / n_assets] * n_assets)

    # 不允许做空
    bounds = [(0, 1)] * n_assets

    # Optimize the objective function using scipy.optimize.minimize
    result = minimize(objective_function, initial_weights, args=(covariance_matrix,), bounds=bounds, method='SLSQP',constraints=cons)

    # Extract the optimized weights from the result
    weights = dict(zip(asset_names, result.x))

    return weights
